In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv

## Load the file as npz for size reasons

In [2]:
natality = np.load('mat2004_data_file.npz', allow_pickle=True)

In [3]:
print(natality.files)

us_natality2004 = natality['X']

['X']


In [4]:
# Display the first few rows of the data frame
print(type(us_natality2004))
print(us_natality2004[:5])

<class 'numpy.ndarray'>
[['S' '1' '2004' ... '' '' '']
 ['S' '1' '2004' ... '' '' '']
 ['S' '1' '2004' ... '' '' '']
 ['S' '1' '2004' ... '' '' '']
 ['S' '1' '2004' ... '' '' '']]


In [6]:
print(us_natality2004.shape)

(4118907, 337)


In [27]:
# OCNTYFIPS is column index 7 which identifies the county in the state
# column index 7 gives county code within state
# column index 5 goves the state where the birth occurred - OSTATE
# to get full identifier, need state and county code

print(us_natality2004[:,7])

['20' '999' '20' ... '47' '81' '81']


## Code to associate a county name to a state abbreviation and county code

In [ ]:
state_col = us_natality2004[:, 5]  # column with state abbreviation
county_col = us_natality2004[:, 7]  # column with county code

# Make the county codes zero-padded strings
county_codes = np.char.zfill(county_col.astype(str), 3)
state_abbrs = np.char.upper(state_col.astype(str))

In [14]:
# Load full county FIPS-to-name mapping
county_df = pd.read_csv(
    "https://raw.githubusercontent.com/kjhealy/fips-codes/master/county_fips_master.csv",
    dtype={"state_fips": str, "county_fips": str, "fips": str},
    encoding="iso-8859-1"
)

In [21]:
# Load the FIPS-to-county-name map
fips_df = pd.read_csv(
    "https://raw.githubusercontent.com/kjhealy/fips-codes/master/county_fips_master.csv",
    #dtype={'state': str, 'county_fips': str},
    #usecols=['state', 'county_fips', 'county_name'],
    dtype={'state': str, 'fips': str, 'county': str},
    usecols=['state', 'fips', 'county'],  
    encoding="iso-8859-1"
)

fips_df = fips_df.rename(columns={'fips': 'county_fips', 'county': 'county_name'})

In [22]:
# make a dictionary for (state_abbr, county_code) -> county_name
county_dict = {
    (row.state.upper(), row.county_fips.zfill(3)): row.county_name
    for row in county_df.itertuples()
}


AttributeError: 'float' object has no attribute 'upper'

In [ ]:
# Vectorized lookup using list
county_names = np.array([
    county_dict.get((state, county), 'Unknown County')
    for state, county in zip(state_abbrs, county_codes)
])

In [ ]:
#new dataset that includes the county name 
us_natality_with_county = np.column_stack((us_natality2004, county_names))


In [17]:
air_quality_by_county = pd.read_csv('annual_aqi_by_county_2004.csv')

print(air_quality_by_county.head())

     State   County  Year  Days with AQI  Good Days  Moderate Days  \
0  Alabama  Baldwin  2004            271        158             98   
1  Alabama  Barbour  2004             49         11             37   
2  Alabama     Clay  2004            283        167            112   
3  Alabama  Colbert  2004            276        178             98   
4  Alabama   DeKalb  2004            361        228            126   

   Unhealthy for Sensitive Groups Days  Unhealthy Days  Very Unhealthy Days  \
0                                   15               0                    0   
1                                    1               0                    0   
2                                    4               0                    0   
3                                    0               0                    0   
4                                    7               0                    0   

   Hazardous Days  Max AQI  90th Percentile AQI  Median AQI  Days CO  \
0               0      143      

In [18]:
# Figuring out how big these files are 
print(us_natality2004.shape)
print(air_quality_by_county.shape)

(4118907, 337)
(1121, 18)


Some useful column codes for health conditions of interest:

Gestational Hypertension: RF_GHYPE
Hypertension Eclampsia: RF_EHYPE
Delivery Method Recode: DMETH_REC
Combined Gestation Recode 3: GESTREC3 (preterm or not)
Birth Weight – Detail in Grams: DBWT (9999 means not reported)